# Decision Tree/Forest Implementation


###  Imports and Preliminaries

In [20]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

from IPython.display import display
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits

In [21]:
class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature # index of feature which is chosen for split
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

###  Decision Tree

In [22]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D) + 2) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        stack = [self.root]

        n_min = 20 # termination criterion: don't split if node contains fewer instances
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                features = np.random.choice(np.arange(D), D_try, replace=False)
                left, right = make_decision_split_node(node, features)
                # Case if all features for all elements in node.data are equal
                if(left == None or right == None):
                    make_decision_leaf_node(node)
                    continue
                stack.append(left)
                stack.append(right)
                node.left = left
                node.right = right
            else:
                make_decision_leaf_node(node)
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        return leaf.output

In [23]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    left, right, = Node(), Node()
    minimal_gini_impurity = None  #  holds performance for the best split
    best_split_l_data, best_split_r_data, best_split_l_labels, best_split_r_labels = [], [], [], []
    
    
    # find best feature j (among 'feature_indices') and best threshold t for the split among all candidates
    
    for f in feature_indices:
        # sort data
        sorted_ind = np.argsort(node.data[ : , f])
        node.data = node.data[tuple([sorted_ind])]
        node.labels = node.labels[tuple([sorted_ind])]
        
        
        last_threshold = node.data[0][f]
        for nd in range(n - 1):
            x = (node.data[nd + 1][f] + node.data[nd][f])/2  #  one threshold candidate per datapoint
            
            if x == last_threshold:
                continue
            last_threshold = x
            
            data_left, data_right, labels_left, labels_right = [], [], [], []
            
            for ind, train in enumerate(node.data):
                if train[f] <= x:
                    data_left.append(train)
                    labels_left.append(node.labels[ind])
                else:
                    data_right.append(train)
                    labels_right.append(node.labels[ind])
            
                      
            data_left, data_right = np.array(data_left), np.array(data_right)
            labels_left, labels_right = np.array(labels_left), np.array(labels_right)
                    
            impurity_left = 0
            impurity_right = 0
            
            _, countL = np.unique(labels_left, return_counts=True)  
            
            #  get the number of labels of each type in left and right child bins
            
            _, countR = np.unique(labels_right, return_counts=True)
            
            # Impurity left
            for count in countL:  #  calculate Gini Impurity
                impurity_left -= np.square(count)
            impurity_left *= np.square(1/data_left.shape[0])
            impurity_left += 1
            
            # Impurity right
            if(data_right.shape[0] != 0): 
                for count in countR:
                    impurity_right -= np.square(count)
                impurity_right *= np.square(1/data_right.shape[0])
            impurity_right += 1
            
            
            gini_impurity_split = impurity_left + impurity_right
            
            if minimal_gini_impurity == None or minimal_gini_impurity > gini_impurity_split:
                minimal_gini_impurity = gini_impurity_split
                node.feature = f
                node.threshold = x
                left.data, right.data, left.labels, right.labels = data_left, data_right, labels_left, labels_right
    if(minimal_gini_impurity == None):
        left, right = None, None
    return left, right 

In [24]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    labels, counts = np.unique(node.labels, return_counts=True)
    node.output = labels[np.argmax(counts)] 

In [25]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    for nd in range(node.labels.shape[0]):
        if node.labels[nd] != node.labels[0]:
            return False
    return True

###  Decision Forest

In [26]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]

    def train(self, data, labels):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            rand_indices = np.random.choice(np.arange(data.shape[0]), data.shape[0], replace=True)
            tree.train(data[tuple([rand_indices])], labels[tuple([rand_indices])])
                       
# x are the features of one number in format: [64] array
    def predict(self, x):
        predictions = np.array([])
        # compute the ensemble prediction
        for tree in self.trees:
            predicted_label = tree.predict(x)
            predictions = np.append(predictions, predicted_label)
        unique, counts = np.unique(predictions, return_counts=True)
        return unique[np.argmax(counts)]

### Decision tree evaluation

In [27]:
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

decision_tree = DecisionTree()
decision_tree.train(data, target)


In [28]:
predictions = []
for i in data:
    predictions.append(decision_tree.predict(i))

In [29]:
def fade_zeros(s):
    return ['color: lightgray' if (v == 0) else 'color: black' for v in s]

numberOfEntries = data.shape[0]
matrix_decision_tree = np.zeros((10, 10))

for i in range(10):
    indices = np.where(target == i)
    predictions = np.array(predictions)
    num, counts = np.unique(predictions[tuple([indices[0]])], return_counts=True)
    counts = counts * (100/ indices[0].shape[0])
    for ind, count in enumerate(counts):
        matrix_decision_tree[i][int(num[ind])] = count
        
        
display(
    pd.DataFrame(data = matrix_decision_tree, index = target_names, columns = target_names)
    .rename_axis('k = ' + str(numberOfEntries), axis = 'columns')
    .style.apply(fade_zeros)
    .format('{0:.2f}%')
)

k = 1797,0,1,2,3,4,5,6,7,8,9
0,94.94%,0.00%,0.00%,0.56%,2.81%,0.00%,0.00%,0.00%,0.00%,1.69%
1,0.00%,97.80%,0.00%,0.55%,0.55%,1.10%,0.00%,0.00%,0.00%,0.00%
2,0.00%,1.69%,89.27%,2.82%,1.13%,1.69%,1.69%,0.00%,1.69%,0.00%
3,0.00%,0.00%,3.28%,89.62%,0.00%,2.19%,2.73%,0.00%,1.64%,0.55%
4,0.00%,1.10%,0.00%,0.00%,95.58%,1.10%,0.00%,0.55%,0.00%,1.66%
5,0.00%,0.55%,6.59%,1.10%,0.55%,85.71%,2.20%,2.20%,0.55%,0.55%
6,0.00%,0.00%,0.00%,0.55%,0.00%,0.00%,98.90%,0.00%,0.00%,0.55%
7,0.00%,0.00%,0.00%,0.00%,3.91%,3.91%,0.00%,91.62%,0.00%,0.56%
8,0.00%,0.00%,1.72%,0.57%,0.00%,2.87%,2.30%,0.00%,91.95%,0.57%
9,0.00%,0.56%,2.22%,1.67%,1.11%,3.89%,1.67%,1.67%,3.89%,83.33%


<span style="color: green"> Above: Confusion Matrix for decision tree classifier. 

<span style="color: green"> Column y of row x contains the percentage of occaisons on which an image depicting digit x was classified as depicting digit y.</span>

This classifier uses only a single decision tree to make its decision. 

**Observations:**

1. Classification accuracy ranges between 85 to 100% depending on the digit. This is decent, but not great. 

2. The most common misclassifications are (true label ~ prediction) 5 ~ '2' and 3 ~ '8'

2. Classification improves if we take more features into account when choosing among bin splits, with the downside of increased training time taken. 


###  Decision Forest Evaluation

In [30]:
digits = load_digits()
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

NTrees = 20
# 1 instance of decision forest with NTrees trees
decision_forest = DecisionForest(NTrees)
decision_forest.train(data, target)

In [32]:
predictions_decision = []

for i in data:
    predictions_decision.append(decision_forest.predict(i))
    
matrix_decision_forest = np.zeros((10, 10))

for i in range(10):
    indices = np.where(target == i)
    predictions_decision = np.array(predictions_decision)
    num, counts = np.unique(predictions_decision[tuple([indices[0]])], return_counts=True)
    counts = counts * (100/ indices[0].shape[0])
    for ind, count in enumerate(counts):
        matrix_decision_forest[i][int(num[ind])] = count
        
numberOfEntries = data.shape[0]

display(
    pd.DataFrame(data = matrix_decision_forest, index = target_names, columns = target_names)
    .rename_axis('k = ' + str(numberOfEntries) + ', NTrees = ' + str(NTrees), axis = 'columns')
    .style.apply(fade_zeros)
    .format('{0:.2f}%')
)

"k = 1797, NTrees = 20",0,1,2,3,4,5,6,7,8,9
0,99.44%,0.00%,0.00%,0.00%,0.56%,0.00%,0.00%,0.00%,0.00%,0.00%
1,0.00%,98.35%,0.00%,0.00%,0.00%,1.65%,0.00%,0.00%,0.00%,0.00%
2,0.00%,1.13%,96.05%,0.00%,1.69%,0.56%,0.56%,0.00%,0.00%,0.00%
3,0.00%,0.00%,1.09%,95.63%,0.55%,1.09%,0.00%,0.00%,0.55%,1.09%
4,0.00%,0.00%,0.00%,0.00%,98.90%,1.10%,0.00%,0.00%,0.00%,0.00%
5,0.00%,0.00%,0.00%,0.55%,1.10%,97.80%,0.00%,0.55%,0.00%,0.00%
6,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%
7,0.00%,0.00%,0.00%,0.00%,3.91%,0.00%,0.00%,96.09%,0.00%,0.00%
8,0.00%,0.57%,0.57%,0.57%,0.00%,0.57%,0.00%,0.00%,97.13%,0.57%
9,0.00%,0.56%,1.67%,0.56%,0.56%,1.11%,0.00%,0.56%,0.56%,94.44%


<span style="color: green"> Above: Confusion Matrix for decision forest classifier. </span>

**Observations:** 

1. By increasing the number of trees, we were able to improve the accuracy for each digit, in some cases by over 10% !

2. As with the decision tree, digit 6 is the most accurately recognized. 

3. Increasing the number of trees in the forest improves the accuracy of the classifier, especially for NTrees > 1 (making the decision forest better than a single decision tree). 

4. By using bootstrap sampling to create a decision forest, one can achieve strict improvements in performance without acquiring more data. Great for data-limited scenarios!